In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# Define the ticker symbol
ticker_symbol = 'AAPL'

# Create a Ticker object
ticker = yf.Ticker(ticker_symbol)

# Get the current stock price
current_price = ticker.history(period='1d')['Close'].iloc[-1]

# Get the expiration dates
try:
    expirations = ticker.options
    print("Expiration Dates:", expirations)
    
    # Choose an expiration date
    expiration_date = expirations[0]  # Modify as needed

    # Get options data for the selected expiration date
    options_data = ticker.option_chain(expiration_date)

    # Extract call and put options data
    calls = options_data.calls
    puts = options_data.puts

    # Convert calls and puts to dataframes
    calls_df = pd.DataFrame(calls)
    puts_df = pd.DataFrame(puts)

    # Function to decompose the contract symbol
    def decompose_contract_symbol(contract_symbol):
        try:
            ticker = contract_symbol[:4]
            expiration = f"20{contract_symbol[4:6]}-{contract_symbol[6:8]}-{contract_symbol[8:10]}"
            option_type = 'Call' if contract_symbol[10] == 'C' else 'Put'
            strike_price = int(contract_symbol[11:]) / 1000
            expiration_date = datetime.strptime(expiration, '%Y-%m-%d')
            return ticker, expiration, option_type, strike_price, expiration_date
        except Exception as e:
            print(f"Error decomposing symbol {contract_symbol}: {e}")
            return pd.Series([None, None, None, None, None])

    # Apply the function to the contract symbols
    calls_df[['Ticker', 'Expiration', 'OptionType', 'StrikePrice', 'ExpirationDate']] = calls_df['contractSymbol'].apply(
        lambda x: pd.Series(decompose_contract_symbol(x))
    )
    puts_df[['Ticker', 'Expiration', 'OptionType', 'StrikePrice', 'ExpirationDate']] = puts_df['contractSymbol'].apply(
        lambda x: pd.Series(decompose_contract_symbol(x))
    )

    # Calculate the number of days until expiration
    current_date = datetime.now()
    calls_df['DaysUntilExpiration'] = (calls_df['ExpirationDate'] - current_date).dt.days
    puts_df['DaysUntilExpiration'] = (puts_df['ExpirationDate'] - current_date).dt.days

    # Plot implied volatility across strike prices
    plt.figure(figsize=(12, 6))
    plt.plot(calls_df['StrikePrice'], calls_df['impliedVolatility'], label='Calls', marker='o')
    plt.plot(puts_df['StrikePrice'], puts_df['impliedVolatility'], label='Puts', marker='x')
    plt.axvline(x=current_price, color='r', linestyle='--', label='Current Price')
    plt.xlabel('Strike Price')
    plt.ylabel('Implied Volatility')
    plt.title(f'Implied Volatility across Strike Prices for {ticker_symbol} Options (Expiration: {expiration_date})')
    plt.legend()
    plt.grid(True)
    plt.show()

except Exception as e:
    print(f"Error fetching or processing data: {e}")